In [2]:
#%%
import pandas as pd
import joblib, json
import numpy as np

In [3]:
#%%
##################################################################################################
Train = pd.read_csv("../01_preprocessing_results/preprocessing/T_train_final_objetivo.csv")
Test = pd.read_csv("../01_preprocessing_results/preprocessing/T_test_final_objetivo.csv")
##################################################################################################

In [4]:
X_train = Train.iloc[:, :-1]
y_train = Train.iloc[:, -1].to_numpy(dtype=float)

X_test = Test.iloc[:, :-1]
y_test = Test.iloc[:, -1].to_numpy(dtype=float)

In [5]:
modelo = joblib.load("../02_lineal_regression_results/regression_lineal/modelo_reg_lineal.pkl")
 

In [6]:
intercepto = modelo.named_steps["linreg"].intercept_
feature_names = modelo.named_steps["dropper"].get_feature_names_out(X_train.columns)
coefs = modelo.named_steps["linreg"].coef_

In [7]:
print(f"intercepto:{intercepto}")
print(f"feature_names:{feature_names}")
print(f"coefs:{coefs}")

intercepto:165.02907511998157
feature_names:['PC1' 'PC2' 'PC3' 'PC4' 'PC5' 'PC6' 'PC7' 'sex___2']
coefs:[ 24.43264545 -24.54137741   5.55415126 -14.59504345  -8.02408632
   7.95602894 -10.24189118 -23.93713619]


In [8]:
# === Betas en espacio post-`pre` (antes del StandardScaler de la PCA) ===

# 1) Cargar el pipeline numérico usado en PCA_full
########################################################################################
pca_pipe = joblib.load("../01_preprocessing_results/preprocessing/pca_pipe_num.joblib")
########################################################################################
pre = pca_pipe.named_steps["pre"]  # ColumnTransformer (pre)
pre

,transformers,"[('num_mean_min', ...), ('num_mean_std', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,False
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'mean'
,fill_value,None


In [9]:
scaler = pca_pipe.named_steps["std_for_pca"]  # StandardScaler
scaler

,copy,True
,with_mean,True
,with_std,True


In [10]:
pca = pca_pipe.named_steps["pca"]  # PCA(n_components=k)
pca

,n_components,7
,copy,True
,whiten,False
,svd_solver,'full'
,tol,0.0
,iterated_power,'auto'
,n_oversamples,10
,power_iteration_normalizer,'auto'
,random_state,0


In [11]:
# 2) Extraer objetos necesarios
pre_feature_names = pre.get_feature_names_out()  # nombres de las p numéricas post-`pre`
pre_feature_names

array(['s2', 's3', 'bmi', 's4', 's1', 's6', 's5', 'age', 'bp'],
      dtype=object)

In [12]:
mu = scaler.mean_  # medias de las p numéricas pre-`pre`
mu

array([ 3.61617749e-01,  3.63636364e-01,  3.58598785e-01,  2.91218626e-01,
        1.08645678e-16,  1.79265369e-16, -4.56311849e-16, -2.33588208e-16,
        1.01040481e-15])

In [13]:
print(sorted(mu, reverse=True))  # medias ordenadas de mayor a menor

[np.float64(0.36363636363636365), np.float64(0.3616177491867392), np.float64(0.35859878463335576), np.float64(0.2912186263980366), np.float64(1.0104048077322526e-15), np.float64(1.7926536911378675e-16), np.float64(1.0864567825077984e-16), np.float64(-2.3358820823917666e-16), np.float64(-4.563118486532754e-16)]


In [14]:
sig = scaler.scale_  # desviaciones estándar de las p numéricas pre-`pre`
sig

array([0.15266234, 0.17222333, 0.18784118, 0.18723359, 1.        ,
       1.        , 1.        , 1.        , 1.        ])

In [15]:
print(sorted(sig))

[np.float64(0.15266234418269284), np.float64(0.17222332852735492), np.float64(0.18723358687482136), np.float64(0.18784118206740613), np.float64(0.9999999999999999), np.float64(1.0), np.float64(1.0), np.float64(1.0), np.float64(1.0)]


In [16]:
C = pca.components_
C  # componentes principales (autovectores) de la PCA

array([[ 0.3610748 , -0.27405673,  0.30772186,  0.42852692,  0.36399198,
         0.3214709 ,  0.3912201 ,  0.2207619 ,  0.2802983 ],
       [ 0.50039061,  0.49231776, -0.31537739, -0.06063795,  0.57566201,
        -0.15322504, -0.10431679,  0.0494591 , -0.18947229],
       [-0.22123162,  0.46434072,  0.10986095, -0.39400314, -0.00386295,
         0.22742945,  0.04620822,  0.53024559,  0.48280606],
       [-0.03481558, -0.29602583, -0.49942891,  0.18715072, -0.14092715,
        -0.12354906, -0.01591306,  0.7378353 , -0.21661711],
       [-0.17117144,  0.0973216 , -0.44637067, -0.01892859,  0.00554454,
         0.77061107,  0.25318002, -0.20394906, -0.24911095],
       [-0.21328911,  0.01673196, -0.43965541,  0.05243267,  0.08942259,
        -0.41930204,  0.58432132, -0.23024337,  0.42417887],
       [ 0.23779422, -0.17437817, -0.38546769,  0.10931169, -0.0955858 ,
         0.18960641, -0.5616061 , -0.16901017,  0.60287535]])

In [17]:
B = pca.explained_variance_
B  # valores propios (autovalores) de la PCA

array([3.93366058, 1.37078544, 1.26765811, 0.70304413, 0.63390095,
       0.53279953, 0.49829589])

In [18]:
#%%
# 3) Aislar betas de Componentes Principales del modelo (según prefijo)
pc_prefix = "PC"
# Creación de la máscara para componentes principales
pc_mask = np.array([c.startswith(pc_prefix) for c in feature_names], dtype=bool)
'''
¿Qué hace?
 
Crea un array booleano que identifica qué características son componentes principales
 
feature_names contiene: ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'sex___2']
 
pc_mask resulta en: [True, True, True, True, True, True, True, False]
 
¿Por qué? El modelo fue entrenado con PCs + variables categóricas, necesitamos separar solo los coeficientes de los PCs.
'''
# Extraccion de coeficientes de componentes principales
beta_PC = coefs[pc_mask]  # (k,)
'''
[ 24.43264545 -24.54137741   5.55415126 -14.59504345  -8.02408632
   7.95602894 -10.24189118]
'''
'''
¿Qué hace?
 
Extrae solo los coeficientes correspondientes a los 7 componentes principales
 
beta_PC tiene forma (7,) con los valores: [24.43, -24.54, 5.55, -14.60, -8.02, 7.96, -10.24]
'''
 
# 4) Transformacion inversa: PCs -> post-`pre` (Variables Estandarizadas
gamma = C.T @ beta_PC
'''[ -6.93778367 -10.74039297  27.18934939   6.48784124  -1.5529309
   3.21976253  20.9767666   -2.10800547  16.54053732]
'''
'''
Matemáticamente:
 
C es la matriz de componentes (7×9)
 
C.T es su transpuesta (9×7)
 
beta_PC es el vector de coeficientes (7×1)
 
gamma resulta en un vector (9×1)
 
¿Qué representa?
 
Convierte los coeficientes del espacio de componentes principales de vuelta al espacio de variables estandarizadas
 
Fórmula: γ = C^T × β_PC
'''
 
# Transformación: Variables estandarizadas → Variables originales
alpha_pre = gamma / sig  # betas por cada variable numérica post-`pre` (p,)
'''
¿Qué hace?
 
Convierte coeficientes de variables estandarizadas a variables en escala original
 
sig contiene las desviaciones estándar usadas en la estandarización
 
Fórmula: α = γ / σ
'''
#Ajuste del Intercepto
beta0_pre = intercepto - np.sum(gamma * (mu / sig))  # intercepto en el mismo espacio
 
'''
¿Por qué es necesario?
 
El intercepto original fue calculado con variables estandarizadas
 
Necesitamos ajustarlo para variables en escala original
 
Fórmula: β₀_nuevo = β₀_original - Σ(γᵢ × μᵢ/σᵢ)
'''
'''
Flujo completo de la transformación:
Variables Originales → Estandarizadas → PCA → Modelo → Coeficientes PCs
                                                            ↓
Variables Originales ← Estandarizadas ← PCA ← Transformación Inversa
'''
 
# 5) Empaquetar resultados
# Betas despues del Post Procesamiento
betas_post_pre_df = pd.DataFrame({
    "feature_post_pre": pre_feature_names,
    "beta_post_pre": alpha_pre
})
########################################################################################################
with open("../01_preprocessing_results/preprocessing/pca_metadata.json", "r", encoding="utf-8") as f:
#######################################################################################################
    meta = json.load(f)
 
orden_original_num = meta["cols_num"]  # orden original de columnas numéricas
presentes = set(betas_post_pre_df["feature_post_pre"].tolist())
orden_final = [c for c in orden_original_num if c in presentes]
 
betas_post_pre_df = (
    betas_post_pre_df
    .set_index("feature_post_pre")
    .loc[orden_final]
    .reset_index()
)
 
print("\nIntercepto (post `pre`):", beta0_pre)
print("\nBetas numéricas (post `pre`, en orden original):")
print(betas_post_pre_df)


Intercepto (post `pre`): 142.14347017042434

Betas numéricas (post `pre`, en orden original):
  feature_post_pre  beta_post_pre
0              age      -2.108005
1              bmi     144.746477
2               bp      16.540537
3               s1      -1.552931
4               s2     -45.445285
5               s3     -62.363171
6               s4      34.651055
7               s5      20.976767
8               s6       3.219763


In [19]:
#%%
# === Predicción en TEST ===
yhat_test = modelo.predict(X_test)

 

# Métricas básicas en test
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import math

 

r2_test   = r2_score(y_test, yhat_test)
rmse_test = math.sqrt(mean_squared_error(y_test, yhat_test))
mae_test  = mean_absolute_error(y_test, yhat_test)

 

print({"R2_test": r2_test, "RMSE_test": rmse_test, "MAE_test": mae_test})

 

# También en train, para comparar generalización
yhat_train = modelo.predict(X_train)
r2_train   = r2_score(y_train, yhat_train)
rmse_train = math.sqrt(mean_squared_error(y_train, yhat_train))
mae_train  = mean_absolute_error(y_train, yhat_train)
print({"R2_train": r2_train, "RMSE_train": rmse_train, "MAE_train": mae_train})



{'R2_test': 0.41750586480401497, 'RMSE_test': 55.763985890343875, 'MAE_test': 45.465992213943466}
{'R2_train': 0.5350070308351939, 'RMSE_train': 53.5027017988126, 'MAE_train': 43.352646819870515}


In [20]:
#%%
# Veredicto
 
# NRMSE y mejora vs media en TEST
std_y_test = float(np.std(y_test, ddof=0))
nrmse_test = rmse_test / (std_y_test + 1e-12)
mejora_pct = 100.0 * (1.0 - nrmse_test)  # % mejor que predecir la media
 
# Semáforo (sin CV), "aquí dentro"
verde   = (r2_test >= 0.70) and (nrmse_test <= 0.50)
amarilo = (0.40 <= r2_test < 0.70) or (0.50 < nrmse_test <= 0.80)
if verde:
    veredicto = "VERDE"
    significado = "confiable para predecir aquí dentro."
elif amarilo:
    veredicto = "AMARILLO"
    significado = "usable con cautela (depende del caso de uso)"
else:
    veredicto = "ROJO"
    significado = "no confiable para predicción aquí dentro"
 
# Generar explicación humanizada automáticamente
def generar_explicacion(r2_test, nrmse_test, mejora_pct, rmse_test, mae_test, veredicto):
    explicacion = []
 
    # Explicación basada en R²
    if r2_test >= 0.8:
        explicacion.append(f"• El modelo explica un {r2_test:.1%} de la variabilidad en los datos, lo que indica un excelente ajuste.")
        explicacion.append("  🎯 **Analogía**: Como un pronóstico del tiempo que acierta 8 de cada 10 días - muy confiable.")
    elif r2_test >= 0.6:
        explicacion.append(f"• El modelo captura un {r2_test:.1%} de la variabilidad, mostrando una buena capacidad predictiva.")
        explicacion.append("  🎯 **Analogía**: Similar a un forecast económico que identifica correctamente las tendencias principales.")
    elif r2_test >= 0.4:
        explicacion.append(f"• Con un R² del {r2_test:.1%}, el modelo tiene capacidad predictiva moderada.")
        explicacion.append("  🎯 **Analogía**: Como un detector de lluvia que funciona bien para saber si lloverá, pero no cuánto.")
    else:
        explicacion.append(f"• El R² de {r2_test:.1%} sugiere que el modelo tiene capacidad predictiva limitada.")
        explicacion.append("  🎯 **Analogía**: Parecido a adivinar el clima lanzando una moneda - mejor que nada, pero poco confiable.")
 
    # Explicación basada en NRMSE
    if nrmse_test <= 0.3:
        explicacion.append(f"• Los errores de predicción son muy bajos ({nrmse_test:.1%} de la variabilidad total).")
        explicacion.append("  📏 **Analogía**: Como medir con una regla milimetrada - alta precisión en las estimaciones.")
    elif nrmse_test <= 0.5:
        explicacion.append(f"• Los errores son moderados ({nrmse_test:.1%} de la variabilidad total).")
        explicacion.append("  📏 **Analogía**: Similar a usar una cinta métrica - útil para la mayoría de propósitos prácticos.")
    elif nrmse_test <= 0.7:
        explicacion.append(f"• Los errores son considerables ({nrmse_test:.1%} de la variabilidad total).")
        explicacion.append("  📏 **Analogía**: Como estimar distancias a ojo - sirve para aproximaciones gruesas.")
    else:
        explicacion.append(f"• Los errores son muy altos ({nrmse_test:.1%} de la variabilidad total).")
        explicacion.append("  📏 **Analogía**: Parecido a adivinar el tamaño de algo desde lejos - muy impreciso.")
 
    # Explicación basada en mejora vs media
    if mejora_pct > 50:
        explicacion.append(f"• Es un {mejora_pct:.0f}% mejor que simplemente predecir el promedio, una mejora sustancial.")
        explicacion.append("  🚀 **Analogía**: Como usar GPS vs. solo un mapa de carreteras - mucho más eficiente.")
    elif mejora_pct > 20:
        explicacion.append(f"• Mejora en un {mejora_pct:.0f}% respecto a predecir la media.")
        explicacion.append("  🚀 **Analogía**: Similar a tener indicaciones de tráfico en tiempo real - claramente mejor que sin ellas.")
    elif mejora_pct > 0:
        explicacion.append(f"• Solo un {mejora_pct:.0f}% mejor que predecir el promedio, mejora marginal.")
        explicacion.append("  🚀 **Analogía**: Como tener una brújula en lugar de solo el norte - ayuda, pero no demasiado.")
    else:
        explicacion.append("• No mejora respecto a predecir el valor promedio.")
        explicacion.append("  🚀 **Analogía**: Como intentar navegar sin brújula ni mapa - no aporta ventaja.")
 
    # Comparación entre train y test (si estuvieran disponibles ambos)
    if 'r2_train' in locals():
        sobreajuste = r2_train - r2_test
        if sobreajuste > 0.2:
            explicacion.append(f"• Hay indicios de sobreajuste (R² train: {r2_train:.3f} vs test: {r2_test:.3f}).")
            explicacion.append("  ⚠️ **Analogía**: Como un estudiante que memoriza las respuestas pero no entiende el concepto.")
        elif sobreajuste < 0.05:
            explicacion.append("• El modelo generaliza bien, sin signos evidentes de sobreajuste.")
            explicacion.append("  ✅ **Analogía**: Similar a un atleta que entrena y compite igual de bien - consistente.")
 
    # Interpretación del veredicto con analogías finales
    if veredicto == "VERDE":
        explicacion.append("\n✅ **Conclusión**: El modelo es confiable para hacer predicciones en contextos similares a los datos de prueba.")
        explicacion.append("🎯 **Analogía final**: Como un piloto automático confiable - puedes usarlo para navegar con seguridad.")
    elif veredicto == "AMARILLO":
        explicacion.append("\n⚠️ **Conclusión**: Úsalo con precaución - puede ser útil para identificar tendencias pero no para predicciones precisas.")
        explicacion.append("🎯 **Analogía final**: Como el forecast de fin de semana - útil para planear, pero lleva paraguas por si acaso.")
    else:
        explicacion.append("\n❌ **Conclusión**: Se recomienda revisar las variables o considerar modelos alternativos.")
        explicacion.append("🎯 **Analogía final**: Como un mapa muy antiguo - mejor conseguir uno actualizado o usar otros métodos.")
 
    return "\n".join(explicacion)
 
# Generar la explicación
explicacion_humanizada = generar_explicacion(
    r2_test, nrmse_test, mejora_pct, rmse_test, mae_test, veredicto
)
 
resumen = {
    "R2_test": r2_test,
    "MAE_test": mae_test,
    "RMSE_test": rmse_test,
    "std(y_test)": std_y_test,
    "NRMSE_test": nrmse_test,
    "Mejora_vs_media_%": mejora_pct,
    "Veredicto": veredicto,
    "significado": significado
}
 
df_resumen = pd.DataFrame(resumen, index=[0]).T
df_resumen.columns = ["Valor"]
df_resumen.index.name = "Métrica"
 
print(df_resumen)
print("\n" + "="*60)
print("EXPLICACIÓN DEL VEREDICTO:")
print("="*60)
print(explicacion_humanizada)
 
def explicacion_breve(r2_test, nrmse_test, mejora_pct, veredicto):
    base = f"Con un R² de {r2_test:.3f} y un error relativo (NRMSE) de {nrmse_test:.3f}, "
 
    # Añadir analogías a la versión breve también
    if veredicto == "VERDE":
        analogia = "Como un GPS confiable - puedes seguir sus indicaciones con seguridad."
        return base + f"el modelo es robusto y explica bien los patrones en los datos, siendo {mejora_pct:.0f}% mejor que usar promedios simples. {analogia}"
    elif veredicto == "AMARILLO":
        analogia = "Similar a un pronóstico de lluvia - útil para planear, pero lleva sombrilla por si acaso."
        return base + f"el modelo tiene capacidad predictiva limitada ({mejora_pct:.0f}% mejor que promedios), adecuado para análisis exploratorios pero no para decisiones críticas. {analogia}"
    else:
        analogia = "Como un mapa desactualizado - mejor buscar herramientas más precisas."
        return base + f"el modelo no supera significativamente las predicciones básicas ({mejora_pct:.0f}% mejora), recomendando revisar el enfoque. {analogia}"
 
print("\n" + "🔍 RESUMEN INTERPRETATIVO:")
print("="*40)
print(explicacion_breve(r2_test, nrmse_test, mejora_pct, veredicto))

                                                          Valor
Métrica                                                        
R2_test                                                0.417506
MAE_test                                              45.465992
RMSE_test                                             55.763986
std(y_test)                                           73.064771
NRMSE_test                                             0.763213
Mejora_vs_media_%                                     23.678697
Veredicto                                              AMARILLO
significado        usable con cautela (depende del caso de uso)

EXPLICACIÓN DEL VEREDICTO:
• Con un R² del 41.8%, el modelo tiene capacidad predictiva moderada.
  🎯 **Analogía**: Como un detector de lluvia que funciona bien para saber si lloverá, pero no cuánto.
• Los errores son muy altos (76.3% de la variabilidad total).
  📏 **Analogía**: Parecido a adivinar el tamaño de algo desde lejos - muy impreciso.
• Mejora en u

In [21]:
#%%
import os, zipfile, glob
 
# Carpeta destino en tu PC
dst_dir = r"mi_regresion_lineal"
os.makedirs(dst_dir, exist_ok=True)
zip_path = os.path.join(dst_dir, "mi_reg_lin_artifacts_bundle.zip")
 
# Archivos que quieres incluir (ajusta si te falta alguno)
candidates = [
    "modelo_reg_lineal.pkl",
    "expected_columns.json",
]
 
present = [f for f in candidates if os.path.exists(f)]
# Si quieres incluir una carpeta (p. ej., 'sample_data'), descomenta:
# for root, _, files in os.walk("sample_data"):
#     for f in files:
#         present.append(os.path.join(root, f))
 
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for f in present:
        zf.write(f, arcname=os.path.basename(f))  # guarda sin subcarpetas
 
print("ZIP creado en:", zip_path)
print("Incluidos:", present)

ZIP creado en: mi_regresion_lineal\mi_reg_lin_artifacts_bundle.zip
Incluidos: []
